# 검토 프로세스
- Google Alert으로 경쟁사 뉴스를 수집한다.

- IFTTT를 활용하여 수집된 뉴스를 Google Sheet에 저장(모음)

- Google API 활용하여 Python 으로 Google Sheet 데이터 불러온다.(service_account.json 파일 생성)

- 대상 google sheet 우상단 share 버튼 클릭후, 서비스계정 이메일을 추가한다.

- Bert 모델 불러온후, Fine tunning 한다.

- Best Model을 활용하여 경쟁사 뉴스에 대해 감정 분류를 한다.

# Get Data from GoogleSheet

In [1]:
import gspread

In [2]:
sa = gspread.service_account("./apikey/news-sentiment-410622-343461f18c24.json")
sh = sa.open("한화오션")  #"삼성중공업" 도 가능
wks = sh.worksheet("Sheet1")
list_data = wks.get_all_values()
list_data[-2:]

[['March 28, 2024 at 01:38AM',
  '한화솔루션, 이사회 의장에 첫 사외이사 선임 - 문화저널21',
  'https://www.mhj21.com/160140',
  '... (한화솔루션), 조지 P. 부시(<b>한화오션</b>) 등 외국인 사외이사를 선임하며 이사회 다양성과 전문성 확보를 위해 노력해왔다. 한화그룹 측은 &quot;한화솔루션의 첫&nbsp;...',
  'Google Alert - 한화오션',
  'https://www.google.com/alerts/feeds/03846553419935279766/11817066535797571903'],
 ['March 28, 2024 at 04:02AM',
  "한국형 차기 구축함, 방산물자 지정…HD현대重·한화오션 '수주전' - 네이트 뉴스",
  'https://news.nate.com/view/20240327n26903?mid=n0200',
  'KDDX의 기본설계를 했던 HD현대중공업이 상세설계 및 선도함 건조 사업을 맡게 될지, 아니면 <b>한화오션</b>이 참가하는 경쟁 구도가 만들어질지 주목된다. 27일&nbsp;...',
  'Google Alert - 한화오션',
  'https://www.google.com/alerts/feeds/03846553419935279766/11817066535797571903']]

# Dataframe 변환후 전처리

In [3]:
import pandas as pd
columns = ['Timestamp', 'Title', 'URL', 'Content', 'Source', 'Source URL']
df = pd.DataFrame(list_data, columns=columns)
df.head()

,Timestamp,Title,URL,Content,Source,Source URL
0,"March 06, 2024 at 09:11PM","[보도국 지금] 특수선 양강 구도 한화오션-HD현대중공업, 갈등 거세져 - 헬로tv뉴스",http://news.lghellovision.net/news/articleView...,[앵커]약 8조 원 규모의 사업인 한국형 차기 구축함 KDDX를 두고 특수선 사업 ...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...
1,"March 06, 2024 at 09:16PM",한화오션 “현대重 임원 기밀유출 개입 정황… 엄중 수사를” - 경남신문,https://m.knnews.co.kr/mView.php?idxno=1427195...,<b>한화오션</b>은 6일 거제시청 브리핑룸에서 한국형 차기구축함(KDDX) 사업...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...
2,"March 06, 2024 at 09:18PM","7조 원대 구축함 수주전…한화오션, HD현대 고발 '확전' / KBS 2024.03...",https://www.youtube.com/watch?v=GnvQyft9x_E,[앵커] <b>한화오션</b>이 경쟁사인 HD현대중공업 임원을 경찰에 고발했습니다....,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...
3,"March 06, 2024 at 09:50PM",중국과 경쟁하기 바쁜데…우리 조선업계는 '전쟁 중'?,https://news.kbs.co.kr/news/view.do?ncd=7906866,<b>한화오션</b>은 즉각 반발했습니다. 그제(4일) 경찰청 국가수사본부에 HD현...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...
4,"March 06, 2024 at 10:07PM","정규직 용접공 8700만원, 하청 4500만원… 위기 때마다 하청만 깎여 - 조선일보",https://www.chosun.com/national/national_gener...,... 하청 35년 차 용접공의 한숨 경남 거제의 <b>한화오션</b>옛 대우조선해...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...


In [4]:
def convert_datetime(date_string):
  datetime_value = pd.to_datetime(date_string, format='%B %d, %Y at %I:%M%p')
  return datetime_value

In [5]:
df["Timestamp"] = df["Timestamp"].apply(convert_datetime)

In [6]:
df['date'] = df["Timestamp"].dt.date
df['month'] = df["Timestamp"].dt.month

In [7]:
df.tail(2)

,Timestamp,Title,URL,Content,Source,Source URL,date,month
1325,2024-03-28 01:38:00,"한화솔루션, 이사회 의장에 첫 사외이사 선임 - 문화저널21",https://www.mhj21.com/160140,"... (한화솔루션), 조지 P. 부시(<b>한화오션</b>) 등 외국인 사외이사를...",Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-03-28,3
1326,2024-03-28 04:02:00,"한국형 차기 구축함, 방산물자 지정…HD현대重·한화오션 '수주전' - 네이트 뉴스",https://news.nate.com/view/20240327n26903?mid=...,KDDX의 기본설계를 했던 HD현대중공업이 상세설계 및 선도함 건조 사업을 맡게 될...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-03-28,3


In [8]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jongb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jongb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
add_stopwords = ["", ".", "...", "·", "for", "to", "with", "is", "%", "(", ")", "?", "[", "]", "{", "}", ",", ";", ":", "-", "./", "/", "”","’", "*", "&", "“", "``", "'",
                "내", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", '==', 'etc…', 'p+',
                "kg㎠", "shall", "one", "two", "three"]

In [10]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

def clean_text(
    string:str,
    punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
    stop_words=['the', 'a', 'and', 'is', 'be', 'will']) -> str:
    """
    A method to clean text
    """
    global add_stopwords
    stop_words_list = stopwords.words('english')
    stop_words_list = stop_words_list + add_stopwords

    # Cleaning the urls
    string = re.sub(r'https?://\S+|www\.\S+', '', string)

    # Cleaning the html elements
    string = re.sub(r'<.*?>', '', string)

    # 한글 제거
    # string = re.sub(r'[가-힣]+', '', string)

    # 텍스트내 숫자만 제거
    # string1=""
    # for t in string:
    #     t1 = re.sub(r'\b\d+\b', '', t)
    #     string1 += t1
    # string = string1

    # Removing the punctuations
    for x in string.lower():
        # x = re.sub(r'\b\d+\b', '', x)
        if x in punctuations:
            string = string.replace(x, "")

    # Converting the text to lower
    string = string.lower()

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()


    return string

In [11]:
df["Content"] = df["Content"].apply(clean_text)

In [12]:
df.tail()

,Timestamp,Title,URL,Content,Source,Source URL,date,month
1322,2024-03-27 22:30:00,"[MTN머니로 상한가 포착] MS와 AI기반 사이버보안 기술 개발 헙무협약, 샌즈랩 급등",https://news.mt.co.kr/mtview.php?no=2024032713...,머니로 top랭킹에 따르면 현재 종목랭킹은 아래와 같다 종목검색순위 hb솔루션 네오...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-03-27,3
1323,2024-03-28 00:31:00,"막 숨통 트인 조선업계, 노조에 발목 잡힐라 '전정긍긍' - 국민일보",https://www.kmib.co.kr/article/view.asp?arcid=...,한화오션은 지난해 매출액 7조4000억원과 영업손실 1965억원을 기록했다 hd현대...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-03-28,3
1324,2024-03-28 00:46:00,"""고령층 뇌 질환 보험금 청구, 10년새 3배↑···30∙40대는 부정맥 주의해야""...",https://www.4th.kr/news/articleView.html?idxno...,연령대별 뇌혈관질환 발생고객 추이사진=한화생명 포쓰저널=박소연 10 한화오션 권혁웅...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-03-28,3
1325,2024-03-28 01:38:00,"한화솔루션, 이사회 의장에 첫 사외이사 선임 - 문화저널21",https://www.mhj21.com/160140,한화솔루션 조지 p 부시한화오션 등 외국인 사외이사를 선임하며 이사회 다양성과 전문...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-03-28,3
1326,2024-03-28 04:02:00,"한국형 차기 구축함, 방산물자 지정…HD현대重·한화오션 '수주전' - 네이트 뉴스",https://news.nate.com/view/20240327n26903?mid=...,kddx의 기본설계를 했던 hd현대중공업이 상세설계 및 선도함 건조 사업을 맡게 될...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-03-28,3


# Load Pre-Trained LLM

In [13]:
import os
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import urllib.request
import seaborn as sns
import matplotlib.pyplot as plt
# import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss
# from tensorflow import keras

In [14]:
from transformers import AutoTokenizer,  AutoModelForSequenceClassification
MODEL_NAME = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
  print("GPU 작동 중")
  mirrored_strategy = tf.distribute.MirroredStrategy()
else:
  print("GPU 미작동 중")

GPU 미작동 중


# Fine tuinig Data 준비

In [17]:
# fine-tunning 데이터 준비
DATASET_URL = "https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv"
DATASET_NAME = "finance_data.csv"

In [18]:
urllib.request.urlretrieve(DATASET_URL,
                           filename = DATASET_NAME
                           )


('finance_data.csv', <http.client.HTTPMessage at 0x1b5500fcd60>)

In [19]:
dataset = pd.read_csv(DATASET_NAME)
dataset.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [20]:
del dataset['sentence']

In [21]:
dataset['labels'] = dataset['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])
dataset.head()

C:\Users\jongb\AppData\Local\Temp\ipykernel_14496\705710288.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['labels'] = dataset['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])


,labels,kor_sentence
0,0,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [22]:
# 중복 데이터 확인
dataset[dataset['kor_sentence'].duplicated()]

,labels,kor_sentence
79,1,텔레콤월드와이어-2006년 4월 7일-TJ 그룹은 모닝 디지털 디자인 Oy 핀란드 ...
789,0,그룹의 사업은 스포츠의 광범위한 포트폴리오와 모든 주요 시장에서의 입지에 의해 균형...
1099,0,이 발표 내용에 대한 책임은 전적으로 발행자에게 있습니다.
1394,0,"핀란드 헬싱키에 본사를 둔 레민카이넨 그룹은 토목 공학, 건축 계약, 기술 건축 서..."
1416,0,"이 보고서는 블랙 앤 데커, 피스카스, 피스카스 브랜드, 후스크바르나 아웃도어 프로..."
2396,0,알스트롬의 주가는 나스닥 OMX 헬싱키에서 인용되고 있다.
2567,0,SSH 통신 보안 코퍼레이션은 핀란드 헬싱키에 본사를 두고 있다.
2889,0,재정적인 세부사항은 공개되지 않았다.
2890,0,재정적인 세부사항은 공개되지 않았다.
2892,0,금융 조건은 공개되지 않았다.


In [23]:
from datasets import Dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split

def df_to_hg_dataset_dict(df):
    '''
    SFT(Supervised Fine-Tuning을 위해 Pandas df를 Huggingface Dataset_dict으로 변환하는 함수
    df columns --> "text", "label"
    '''
    train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
    
    train_dataset = Dataset.from_pandas(train_df)
    train_dataset = train_dataset.remove_columns(["__index_level_0__"])

    test_dataset = Dataset.from_pandas(test_df)
    test_dataset = test_dataset.remove_columns(["__index_level_0__"])

    dataset_dict = DatasetDict({'train': train_dataset, 'test': test_dataset})
    return dataset_dict

In [24]:
dataset1 = df_to_hg_dataset_dict(dataset)
dataset1

DatasetDict({
    train: Dataset({
        features: ['labels', 'kor_sentence'],
        num_rows: 3392
    })
    test: Dataset({
        features: ['labels', 'kor_sentence'],
        num_rows: 1454
    })
})

In [25]:
def tokenize_function(examples):
    return tokenizer(examples["kor_sentence"], padding="max_length", truncation=True)

In [26]:
tokenized_datasets = dataset1.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3392 [00:00<?, ? examples/s]

Map:   0%|          | 0/1454 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'kor_sentence', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3392
    })
    test: Dataset({
        features: ['labels', 'kor_sentence', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1454
    })
})

In [27]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(50))
small_train_dataset.shape, small_eval_dataset.shape

((100, 5), (50, 5))

# Fine Tuning - PEFT + LORA

In [28]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [29]:
# from transformers import TrainingArguments
# output_dir="test_trainer1"
# training_args = TrainingArguments(output_dir=output_dir)

In [30]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

function 'cadam32bit_grad_fp32' not found


D:\GG_env\llm_env\lib\site-packages\bitsandbytes\cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [31]:
peft_config = LoraConfig(task_type="SEQ_CLS", # Sequence Classification.
                        r=4,  # Intrinsic rank of trainable weight matrix.
                        lora_alpha=32,  # similar to Learning rate.
                        lora_dropout=0.01, # probability of dropout nodes.
                        target_modules = ['query']) # LoRA is applied to the query layer.

In [32]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 76,035 || all params: 110,695,686 || trainable%: 0.06868831365298192


In [33]:
# Hyperparameters
lr = 1e-3 # size of optimization step
batch_size = 4 # No.of.examples processed per optimization step
num_epochs = 3 # No.of.times the model runs through training data.
output_dir="test_trainer1"
# training arguments
training_args = TrainingArguments(
    output_dir= output_dir,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [34]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [35]:
trainer = Trainer(
    model=model, # Our PEFT Model, here it is distilbert
    args=training_args, # Hyperparamaters
    train_dataset=small_train_dataset, # training data
    eval_dataset=small_eval_dataset, # validation data
    tokenizer=tokenizer, # tokenizer
    data_collator=data_collator, # dynamic sequence padding
    compute_metrics=compute_metrics,  # model perfomance evaluation metric
)


In [36]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.912222,0.600000
2,No log,0.806083,0.720000
3,No log,0.791480,0.740000


TrainOutput(global_step=75, training_loss=0.7543916829427083, metrics={'train_runtime': 981.5656, 'train_samples_per_second': 0.306, 'train_steps_per_second': 0.076, 'total_flos': 79004099174400.0, 'train_loss': 0.7543916829427083, 'epoch': 3.0})

In [37]:
metrics = trainer.evaluate(eval_dataset=small_eval_dataset)
metrics

{'eval_loss': 0.7914796471595764,
 'eval_accuracy': 0.74,
 'eval_runtime': 69.2544,
 'eval_samples_per_second': 0.722,
 'eval_steps_per_second': 0.188,
 'epoch': 3.0}

In [38]:
# Save the fine-tuned model
trainer.model.save_pretrained(output_dir)

# Load Fine Tuned Model and Test

In [42]:
# Load the saved fine-tuned model
ft_model = AutoModelForSequenceClassification.from_pretrained(output_dir, num_labels=3)
ft_model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): lora.Linear(
                (base_layer): Linear(in_features=768, out_features=768, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.01, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=

In [49]:
# text = "한화오션은 지난해 말부터 카타르에너지로부터 lng 운반선 40척 계약을 수주했습니다...	"
# text = "대한민국은 출산율이 너무 낮아서 국가 소멸위기에 처하고 있습니다."
text = "국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."

In [50]:
import torch
inputs = tokenizer(text, return_tensors="pt")
print(inputs)
# Perform inference
with torch.no_grad():
    outputs = ft_model(**inputs)
print(outputs.logits)
# Get the predicted label
predicted_label = torch.argmax(outputs.logits).item()
print(predicted_label)

{'input_ids': tensor([[    2,  3854,  4299,  7138,  3769,  2179,  1424,  2258,  2229,  2073,
          1764,  2466,  4345, 27135,  6106,  8918,  4070,  2069,  7208,  2371,
          4007,    16,  4176,  2079,    18,    18,    18,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])}
tensor([[-0.3889,  0.5740, -0.6841]])
1
